In [1]:
import requests

In [2]:
import json

In [3]:
import base64

In [4]:
import random

In [5]:
import os

In [6]:
from tqdm import tqdm

In [7]:
from PIL import Image, ImageOps

In [8]:
# Convert image to Base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read())

# Call the WD 1.4 Tagger API
def get_tags_from_image(image_path, model="wd14-vit-v2", threshold=0.35):
    reserved_tags = {"general", "sensitive", "questionable", "explicit"}
    url = "http://localhost:7860/tagger/v1/interrogate"  # Replace with your actual host if different
    headers = {"Content-Type": "application/json"}
    base64_image = image_to_base64(image_path)
    payload = {
        "image": base64_image.decode("utf-8"),
        "model": model,
        "threshold": threshold
    }
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        caption = response.json()["caption"]
        result_tags = set()
        for tag in caption:
            if tag not in reserved_tags:
                result_tags.add(tag)
        return result_tags, base64_image
    else:
        print("Error:", response.status_code, response.text)
        return None, None


In [9]:
# Step 2: Format Tags into a Template
def format_tags(tags, additional_tags=set(), remove_tags=set(), prompt_template="{tags}"):
    for tag in remove_tags:
        if tag in tags:
            tags.remove(tag)
    return prompt_template.format(tags=', '.join(tags), additional_tags = ', '.join(additional_tags))


In [10]:
# Step 3 & 4: Configure ControlNet and Hires Fix, and Generate Image
def generate_image(positive_prompt, negative_prompt, input_base64_image, overrides={}, verbose=False):
    url = "http://127.0.0.1:7860/sdapi/v1/txt2img"  # Replace with WebUI API endpoint
    with open("request.json", "rb") as f:
        payload = json.load(f)
    payload["alwayson_scripts"]["ControlNet"]["args"][0]["image"]["image"] = input_base64_image.decode("utf-8")
    payload["prompt"] = positive_prompt
    payload["hr_prompt"] = positive_prompt
    payload["negative_prompt"] = negative_prompt
    payload["hr_negative_prompt"] = negative_prompt

    # Generate a random seed (32-bit unsigned integer)
    if "seed" in overrides:
        seed = overrides[seed]
    else:
        seed = random.randint(0, 2**32 - 1)
    if verbose:
        print(seed)
    payload["seed"] = seed
    payload["alwayson_scripts"]["Seed"]["args"][0] = seed

    for key in overrides:
        payload[key] = overrides[key]

    response = requests.post(url='http://127.0.0.1:7860/sdapi/v1/txt2img', json=payload)
    
    # Decode the Base64 image
    image = response.json()['images'][0]
    image_data = base64.b64decode(image)

    return response, image_data

In [11]:
from io import BytesIO
from PIL import Image

In [12]:
def preview_image(image_data):
    img = Image.open(BytesIO(image_data))
    return img

In [13]:
from IPython.display import display

In [14]:
def preview_image_with_input(image_data, input_image_data):
    # Open the images
    img1 = Image.open(BytesIO(image_data))
    img2 = Image.open(BytesIO(base64.b64decode(input_image_data)))

    # Get dimensions of the first image (image_data)
    target_width, target_height = img1.size

    # Resize img2 while maintaining its aspect ratio
    img2 = ImageOps.contain(img2, (target_width, target_height))

    # # Resize img1 to match the target dimensions exactly
    # img1 = img1.resize((target_width, target_height), ImageResampling.LANCZOS)

    # Create a new image with enough width to fit both images side by side
    combined_width = target_width * 2
    combined_image = Image.new('RGB', (combined_width, target_height))

    # Paste the resized images side by side
    combined_image.paste(img1, (0, 0))
    combined_image.paste(img2, (target_width, 0))

    return combined_image


In [15]:
import time

def save_image(image_data, save_dir="H:/sd-webui-aki-v4.8/log/images", prefix=""):
    """
    Saves an image to the specified directory with an optional prefix in the filename.

    Args:
        image_data (bytes): The image data in bytes format.
        save_dir (str): The directory where the image will be saved.
        prefix (str): An optional prefix for the filename.
    """
    # Get the current timestamp
    timestamp = time.strftime("%Y%m%d_%H%M%S")

    # Open the image from the given byte data
    img = Image.open(BytesIO(image_data))

    # Generate the filename with the prefix and timestamp
    filename = f"{prefix}_{timestamp}.jpg"  # Change extension if needed

    # Save the image to the specified directory with the generated filename
    img.save(f"{save_dir}/{filename}")

In [17]:
POSITIVE_TEMPLATE = "(((masterpiece))),(((best quality))),highres,ultra detailed,(detailed face:1.7),realistic,photorealistic, \
{tags},\
{additional_tags}, "
POSITIVE_TEMPLATE += "<lora:Liyuu:0.8>, \
(black hair:1.8),(small breasts:1.5),(brown eyes:1.3),"
# POSITIVE_TEMPLATE += "<lora:add_detail:1>,"
# POSITIVE_TEMPLATE += "<lora:Lgirl-v6_500steps:-1>,"

In [18]:
NEGATIVE_TEMPLATE = "{tags},(worst quality:2),(low quality:2),(normal quality:2),badhandv4,EasyNegative,EasyNegativeV2,FastNegativeV2,ng_deepnegative_v1_75t,child,(abs:2),(ribs:2),(thick lips:2),(pencil:2),(extra legs:2),"

In [19]:
def full_flow(input_image,
              remove_tags=set(),
              additional_tags=set(),
              negative_additional_tags=set(),
              overrides={},
              generation_batch=1,
              preview=False,
              save=True,
              save_dir=None,
              prefix="",
              contain_original_filename=False,
              verbose=True):
    """
    Processes an image using the full flow pipeline.

    Args:
        input_image (str): Path to the input image.
        remove_tags (set): Tags to remove during processing.
        additional_tags (set): Tags to add during processing.
        negative_additional_tags (set): Negative tags to add during processing.
        overrides (dict): Override parameters for the image generation.
        save (bool): Whether to save the generated image.
        save_dir (str): Directory to save the processed images.
        prefix (str): Prefix for the saved filename.
        contain_original_filename (bool): Use the original filename as the prefix if True.
        verbose (bool): Whether to print logs during processing.
    """
    # Calculate the dimensions of the input image
    with Image.open(input_image) as img:
        original_width, original_height = img.size

    # Determine new dimensions maintaining the aspect ratio
    if original_width > original_height:
        width, height = 768, 512
    else:
        width, height = 512, 768

    tags, input_base64_image = get_tags_from_image(input_image, "wd-swinv2-v3")
    if verbose:
        print(tags)

    positive_prompt = format_tags(tags,
                                  additional_tags,
                                  remove_tags,
                                  prompt_template=POSITIVE_TEMPLATE)
    if verbose:
        print(positive_prompt)

    negative_prompt = format_tags(negative_additional_tags,
                                  set(),
                                  prompt_template=NEGATIVE_TEMPLATE)
    if verbose:
        print(negative_prompt)

    overrides.update({
        "height": height,
        "width": width,
    })

    for i in range(generation_batch):
        if verbose:
            print(f"Generating {i+1}/{generation_batch}...")
        response, image_data = generate_image(positive_prompt, negative_prompt,
                                            input_base64_image, overrides,
                                            verbose)

        if preview:
            preview_image = preview_image_with_input(image_data, input_base64_image)
            display(preview_image)

        if save:
            if contain_original_filename:
                prefix = f"{os.path.splitext(os.path.basename(input_image))[0]}_"
            if save_dir:
                save_image(image_data, save_dir, prefix=prefix)
            else:
                save_image(image_data, prefix=prefix)

        
    return image_data

In [20]:
def call_full_flow_on_network_folder(input_folder,
                                     remove_tags=set(),
                                     additional_tags=set(),
                                     negative_additional_tags=set(),
                                     overrides={},
                                     generation_batch=1,
                                     save=True,
                                     save_dir=None,
                                     contain_original_filename=False,
                                     progress_bar=False,
                                     include_subfolders=False):
    """
    Calls the `full_flow` function on all images within a network folder.

    Args:
        network_folder (str): The path to the network folder containing images.
        remove_tags (set): Tags to remove during processing.
        additional_tags (set): Tags to add during processing.
        negative_additional_tags (set): Negative tags to add during processing.
        overrides (dict): Override parameters for the `full_flow` function.
        save (bool): Whether to save the output of the `full_flow` function.
        save_dir (str): Directory to save the processed images.
        contain_original_filename (bool): Use the original filename as the prefix if True.
        progress_bar (bool): Whether to show a progress bar.
    """
    # Check if the network folder exists
    if not os.path.exists(input_folder):
        raise FileNotFoundError(
            f"The specified folder does not exist: {input_folder}")

    # Get all image files from the folder (and subfolders if include_subfolders is True)
    image_files = []
    for root, _, files in os.walk(input_folder) if include_subfolders else [(input_folder, None, os.listdir(input_folder))]:
        for file_name in files:
            if file_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".gif")):
                image_files.append(os.path.join(root, file_name))

    # Iterate through files with optional progress bar
    for file_name in tqdm(image_files,
                          desc="Processing images",
                          disable=not progress_bar):
        file_path = os.path.join(input_folder, file_name)

        try:
            full_flow(input_image=file_path,
                      remove_tags=remove_tags,
                      additional_tags=additional_tags,
                      negative_additional_tags=negative_additional_tags,
                      overrides=overrides,
                      generation_batch=generation_batch,
                      save=save,
                      save_dir=save_dir if save else None,
                      contain_original_filename=contain_original_filename,
                      verbose=not progress_bar)
        except Exception as e:
            if not progress_bar:
                print(f"Error processing {file_path}: {e}")

In [ ]:
image_path = "H:/gen_photos/inputs/3812.png"  # Replace with your image path
tags, input_base64_image = get_tags_from_image(image_path, "wd-swinv2-v3")
tags

In [392]:
remove_tags = {"large_breasts", "purple_eyes", "teeth", "mixed-sex_bathing", "shared_bathing"}

In [393]:
additional_tags = {
    "(detailed face:2.0)",
}

In [394]:
positive_prompt = format_tags(tags, additional_tags, remove_tags, prompt_template=POSITIVE_TEMPLATE)

In [396]:
negative_additional_tags = {
}

In [397]:
negative_prompt = format_tags(negative_additional_tags, set(), prompt_template=NEGATIVE_TEMPLATE)

In [399]:
overrides = {
    "denoising_strength": 0.3,
    "height":768,
    "width": 512,
    # "height":512,
    # "width": 768,
}

In [ ]:
response, image_data = generate_image(positive_prompt, negative_prompt, input_base64_image, overrides)
# preview_image(image_data)
preview_image_with_input(image_data, input_base64_image)

In [405]:
save_image(image_data)

In [51]:
full_flow(
    input_image=
    r"\\10.0.0.22\home\tddownload\acgbuluo\习呆呆\Adobe Photoshop CS2 Windows 1500x2000_107440.jpg",
    remove_tags={
        "large_breasts",
    },
    additional_tags={
        "(detailed face:2.0)",
    },
    negative_additional_tags={
        "(glasses:2.0)",
        "(mask:2.0)",
        "(eye mask:2.0)",
    },
    overrides={
        "denoising_strength": 0.3,
        "height": 768,
        "width": 512,
    },
    save=True,
    save_dir="./tmp/",
    contain_original_filename=True,
    verbose=True,
    generation_batch=2,
)

{'short_hair', 'realistic', 'underwear', 'solo', 'looking_at_viewer', 'panties', 'ram_(re:zero)', 'maid_headdress', 'bedroom', 'hair_over_one_eye', 'bed', 'breasts', 'rem_(re:zero)', 'thighhighs', 'nude', 'horns', 'hair_ornament', 'small_breasts', 'blue_eyes', 'curtains', 'blue_hair', '1girl', 'x_hair_ornament', 'nipples', 'all_fours', 'indoors'}
(((masterpiece))),(((best quality))),highres,ultra detailed,(detailed face:1.7),realistic,photorealistic, short_hair, realistic, underwear, solo, looking_at_viewer, panties, ram_(re:zero), maid_headdress, bedroom, hair_over_one_eye, bed, breasts, rem_(re:zero), thighhighs, nude, horns, hair_ornament, small_breasts, blue_eyes, curtains, blue_hair, 1girl, x_hair_ornament, nipples, all_fours, indoors,(detailed face:2.0), <lora:baoer2-000032:0.8>, <lora:add_detail:1>, (black hair:1.8),(medium breasts:1.5),<lora:Lgirl-v6_500steps:-2>,
(glasses:2.0), (eye mask:2.0), (mask:2.0),(worst quality:2),(low quality:2),(normal quality:2),badhandv4,EasyNegati

In [55]:
random.randint(0, 2**32 - 1)

2878213471

In [28]:
# input_folder = r"\\10.0.0.22\home\tddownload\acgbuluo\习呆呆"
input_folder = r"H:\gen_photos\inputs\250217inputs"
call_full_flow_on_network_folder(
    input_folder=input_folder,
    include_subfolders=True,
    remove_tags={"large_breasts",},
    additional_tags={
    },
    negative_additional_tags={
        "(facial_mark:2.0)",
        "(forehead_mark:2.0)",
        "(watermark:2.0)",
        "(glasses:2.0)",
        "(mask:2.0)",
        "(eye mask:2.0)",
    },
    overrides={
        "denoising_strength": 0.4,
        # "seed": random.randint(0, 2**32 - 1),
    },
    save=True,
    save_dir="./tmp/43380/",
    contain_original_filename=True,
    progress_bar=True,
    generation_batch=2,
) 

Processing images: 100%|██████████| 144/144 [1:25:44<00:00, 35.73s/it]
